In [3]:
#可理解为kernal窗口在输入数据矩阵上滑动相乘，每次相乘得到一个输出
from mxnet import autograd, nd
from mxnet.gluon import nn

def corr2d(X,K):      #定义kernal与输入输出的算术关系
    h, w = K.shape     #获取kernal的尺寸以计算输出矩阵维度
    Y = nd.zeros((X.shape[0] - h + 1,X.shape[1]- w +1))   #默认步长为1，计算输出矩阵维度
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h,j: j + w]* K).sum()     #计算每个输出的值
    return Y

In [4]:
X = nd.array([[0, 1, 2],[3, 4, 5],[6, 7, 8]])
K = nd.array([[0, 1],[2, 3]])
corr2d(X, K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

In [5]:
class Conv2D(nn.Block):
    def __init__(self, kernal_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=kernal_size)   #自定义模型权重
        self.bias = self.params.get('bias', shape=(1,))         #添加偏差项
        
    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

In [6]:
#尝试边缘检测
X = nd.ones((6,8))
X[:, 2:6] = 0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [7]:
#会在边缘得到1或-1，其余全为0
K = nd.array([[1, -1]]) 
Y = corr2d(X,K)
Y


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [11]:
#通过学习训练迭代优化kernal权重参数
#构造输出通道数为1，卷积核大小为(1,2)
conv2d = nn.Conv2D(1, kernel_size=(1,2))
conv2d.initialize()
#二维卷积层使用四维输入输出，格式为(样本， 通道， 高， 宽)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))   #可根据步长，输入形状，卷积核形状得出

for i in range(10):             #迭代十次优化权重参数
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat - Y)** 2
    l.backward()            
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0:
        print('batch %d,loss %.3f' % (i +1,l.sum().asscalar()))

batch 2,loss 5.063
batch 4,loss 0.864
batch 6,loss 0.151
batch 8,loss 0.028
batch 10,loss 0.006


In [12]:
conv2d.weight.data()


[[[[ 0.9925716 -0.9841616]]]]
<NDArray 1x1x1x2 @cpu(0)>

In [ ]:
#获得结果接近于用于计算Y的kernel